# Lab07-1 Tips

## Maximum Likelihood Estimation (MLE)

## Overfitting
방지하려면
* More Data
* Less features
* Regularization
  * Early Stopping
  * Reducing Network Size
  * Weight Decay
  * Dropout
  * Batch Normalization

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Training and Test Dataset

In [2]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [3]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    
    def forward(self, x):
        return self.linear(x)

In [4]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [5]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)

        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

In [6]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print(f'Accuracy: {correct_count / len(y_test) * 100:.2f}% Cost: {cost.item():.6f}')

In [7]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [8]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.00% Cost: 1.425844


## Learning Rate 
* learning rate가 너무 크면 diverge 하면서 cost가 점점 늘어난다(overshooting).
* 너무 작으면 cost가 거의 줄어들지 않는다.

In [9]:
# learning rate 클 때
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e5)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.812500
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.000000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.250000
Epoch    7/20 Cost: 1532260.125000
Epoch    8/20 Cost: 1651504.000000
Epoch    9/20 Cost: 521878.437500
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.500000
Epoch   13/20 Cost: 1487888.250000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.625000


In [10]:
# learning rate 작을 때
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-10)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


## Data Preprocessing
* Standardization (정규분포화)
$$
x'_j = \frac{x_j - \mu _j}{\sigma_j}
$$

In [11]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [76, 66, 70]])
y_train = torch.FloatTensor([[152], [158], [180], [196], [142]])

In [12]:
# Standardization
mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.2027, -0.3758, -0.8398],
        [ 0.7371,  0.2778,  0.5863],
        [ 0.3492,  0.5229,  0.3486],
        [ 1.0281,  1.0948,  1.1409],
        [-0.9117, -1.5197, -1.2360]])


In [13]:
class MultivariateLinearREgressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [14]:
model = MultivariateLinearREgressionModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [15]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)

        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

In [16]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 27880.556641
Epoch    1/20 Cost: 17781.603516
Epoch    2/20 Cost: 11383.929688
Epoch    3/20 Cost: 7308.788086
Epoch    4/20 Cost: 4706.255371
Epoch    5/20 Cost: 3042.084473
Epoch    6/20 Cost: 1977.246826
Epoch    7/20 Cost: 1295.629272
Epoch    8/20 Cost: 859.177612
Epoch    9/20 Cost: 579.612427
Epoch   10/20 Cost: 400.456940
Epoch   11/20 Cost: 285.573730
Epoch   12/20 Cost: 211.835159
Epoch   13/20 Cost: 164.439499
Epoch   14/20 Cost: 133.914017
Epoch   15/20 Cost: 114.194908
Epoch   16/20 Cost: 101.401382
Epoch   17/20 Cost: 93.049072
Epoch   18/20 Cost: 87.547310
Epoch   19/20 Cost: 83.877892


# Lab07-2 MNIST

In [17]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [19]:
# parameters
training_epochs = 15
batch_size = 100

In [20]:
import torchvision.datasets as dsets

mnist_train = dsets.MNIST(root="MNSIT_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNSIT_data/", train=False, transform=transforms.ToTensor(), download=True)

In [21]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [22]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [23]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [24]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.535150647
Epoch: 0002 cost = 0.359577745
Epoch: 0003 cost = 0.331264287
Epoch: 0004 cost = 0.316404670
Epoch: 0005 cost = 0.307106972
Epoch: 0006 cost = 0.300456554
Epoch: 0007 cost = 0.294933438
Epoch: 0008 cost = 0.290956199
Epoch: 0009 cost = 0.287074089
Epoch: 0010 cost = 0.284515619
Epoch: 0011 cost = 0.281914085
Epoch: 0012 cost = 0.279526860
Epoch: 0013 cost = 0.277636588
Epoch: 0014 cost = 0.275874794
Epoch: 0015 cost = 0.274422765
Learning finished


In [25]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8883000016212463
Label:  8
Prediction:  3


In [34]:
for i in mnist_test.data[r:r + 1].reshape(28, 28):
    for a in i:
        print(f'{a.numpy():3d}', end="")
    print()

  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 27142213188175 23  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 71231252255254254255252  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2 54245244166134 96105251122  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 81254240 62  0  0  4229221  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 47253238130  0  0 29166254241 37  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 49254141  4 2317923525421

In [27]:
#plt.imshow(mnist_test.data[r:r + 1].reshape(28, 28), cmap='Greys')
#plt.show()